# Shor's algorithm in a few lines of numpy

## Logical functions as linear transformations

Implementación de funciones lógicas como transformaciones matriciales entre los espacios completos de configuraciones de entrada y salida.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = [3,3]

def showmat(m):
    plt.imshow(m); plt.axis('off');

In [ ]:
def decode(bits):
    r = np.zeros(2**len(bits),int)
    r[int(''.join([str(b) for b in bits ]),2)] = 1
    return r

def encode(oneshot):
    x = np.argmax(oneshot)
    l = np.round(np.log2(len(oneshot))).astype(int)
    fmt = f"{{x:0{l}b}}"
    return list([int(d) for d in fmt.format(x=x) ])

In [ ]:
import itertools

bit = [0,1]

def bits(n):
    return itertools.product(*[bit]*n)

def Oper(l):
    return np.array([decode(x) for x in l]).T

In [ ]:
def tp(A,B):
    return np.vstack([np.hstack(x) for x in np.tensordot(A,B,axes=0)])

def tps(As):
    if len(As) == 1:
        return As[0]
    else:
        return tp(As[0],tps(As[1:]))

Las puertas lógicas más comunes:

In [ ]:
WH = np.array([[1, 1],
               [1,-1]])/np.sqrt(2)

And = Oper([ [1 if x==1 and y==1 else 0] for x,y in bits(2) ])

Not  = Oper([ [1-x] for x, in bits(1) ])

Id = Not@Not

CNot = Oper([ [x,y if x==0 else 1-y] for x,y in bits(2) ])

Toffoli = Oper([ [x1, x2, 1-y if x1==1 and x2==1 else y] for x1,x2,y in bits(3) ])

## Classical gates

Verificamos que el orden de expansión es consistente.

In [ ]:
list(enumerate(bits(3)))

In [ ]:
list(map(decode,bits(3)))

In [ ]:
list(enumerate(map(encode, map(decode,bits(3)))))

In [ ]:
showmat(Oper([[x,y,z] for x,y,z in bits(3)]))

La función tps hace el tensor product y por tanto actúa como combinación en paralelo.

In [ ]:
for x,y,z in bits(3):
    print(x,y,z,encode(tps([Not,Id,Id])@decode([x,y,z])))

In [ ]:
Test  = Oper([ [x, y, z, z] for x,y,z in bits(3) ])

In [ ]:
showmat(Test)

In [ ]:
for x,y,z in bits(3):
    print(x,y,z,encode(Test@decode([x,y,z])))

Empezamos probando operaciones clásicas:

In [ ]:
plt.figure(figsize=(3,3))
showmat(And)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(And@decode([x,y])))

In [ ]:
Or = Not @ And @ tps([Not,Not])

In [ ]:
plt.figure(figsize=(3,3))
showmat(Or)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(Or@decode([x,y])))

La gracia está en combinar circuitos fijos, expandiendo entradas adecuadamente con tensor products.

## Adder

Construimos un sumador de 4 bits encadenando 4 de 1 bit:

In [ ]:
adder = Oper([( (x+y+s)%2,(x+y+s)//2) for s,x,y in bits(3) ])

In [ ]:
for s,x,y in bits(3):
    print(s,x,y,encode(adder@decode([s,x,y])))

Para conectarlos hay que dejar pasar las entradas y salidas en los canales adecuados en cada etapa:

In [ ]:
step1 = tps([adder,Id,Id,Id,Id,Id,Id])
showmat(step1); plt.show()
step2 = tps([Id,adder,Id,Id,Id,Id])
showmat(step2); plt.show()
step3 = tps([Id,Id,adder,Id,Id])
showmat(step3); plt.show()
step4 = tps([Id,Id,Id,adder])
showmat(step4); plt.show()

In [ ]:
adder4 = step4 @ step3 @ step2 @ step1
print(adder4.shape)
plt.figure(figsize=(8,3))
showmat(adder4)

Utilidades de conversión decimal $\leftrightarrow$ binario:

In [ ]:
def dec(x):
    return sum([v * 2**k for k,v in enumerate(reversed(x))])

In [ ]:
dec([1,1,0])

In [ ]:
def binary(num,length=4):
    fmt = '{:0'+str(length)+'b}'
    return  [int(c) for c in fmt.format(num)]

In [ ]:
binary(6,8)

Construimos la entrada alternando los bits de cada número, con los bits más significativos al final.

In [ ]:
def rev(x): return list(reversed(x))

In [ ]:
a = 8
b = 7
ab = [0]+list(np.array(list(zip(reversed(binary(a)),reversed(binary(b))))).flatten())
ab

In [ ]:
encode(adder4 @ decode(ab))

In [ ]:
c = dec(rev(_))
c, c==a+b

Estos circuitos son deterministas: cada elemento de la base de estados de entrada produce sin ambiguedad una configuración de salida. Cada columna solo tiene un uno. Pero es normal que varios estados de entrada vayan al mismo de salida. Cada fila es un posible resultado, y los unos en ella indican los estados de entrada que lo producen.

In [ ]:
c = 7

pos = dec(rev(binary(c,5)))

In [ ]:
list(np.where(adder4[pos])[0])

In [ ]:
bs = binary(100,9)
print(bs)
bs[0], dec(list(reversed(bs[1::2]))), dec(list(reversed(bs[2::2])))

In [ ]:
bs = binary(280,9)
print(bs)
bs[0], dec(list(reversed(bs[1::2]))), dec(list(reversed(bs[2::2])))

In [ ]:
def permute(js,n):
    join  = Oper( [ [xs[k] for k in js] + [xs[k] for k in range(n) if k not in js] for xs in bits(n)] )
    return join

def extend(gate, tot):
    o,i = gate.shape
    n = round(np.log2(i))
    return tps([gate]+[Id]*(tot-n))

def operateWith(gate,js,n):
    join = permute(js, n)
    return join.T @ extend(gate, n) @ join

$$[x,y,z,0,0] \rightarrow [x,\, y,\, z,\, x \land y,\, x \land y \land z]$$

In [ ]:
encode(operateWith(Toffoli,[2,3,4],5) @ operateWith(Toffoli,[0,1,3],5) @ decode([1,1,1]+[0]*2))

In [ ]:
for x,y,z in bits(3):
    print(encode(operateWith(Toffoli,[2,3,4],5) @ operateWith(Toffoli,[0,1,3],5) @ decode([x,y,z]+[0]*2)))

## Uncertainty

Estas matrices de transformación son un caso particular de las [matrices estocásticas](https://en.wikipedia.org/wiki/Stochastic_matrix), que transforman densidades de probabilidad en densidades de probabilidad. Recordemos que producto matriz vector implementa la contracción P(y) = Sum P(y|x) P(x). Las matrices estocásticas son probabilidades condicionadas, en las que cada columna suma 1. Los circuitos lógicos anteriores son transformaciones deterministas donde las columnas solo tienen un uno. Pero pueden utilizarse sin problema para transformar distribuciones de probabilidad.

Con esta operación construimos un bit completamente incierto:

In [ ]:
erase = np.array([[1,1],
                  [1,1]])/2

In [ ]:
erase @ [0.2, 0.8]

In [ ]:
erase @ erase @ [0.2, 0.8]

En el  sumador anterior metemos un bit incierto:

In [ ]:
probs = adder4 @ tps([Id,erase,Id,Id,Id,Id,Id,Id,Id]) @ decode(ab)
probs

In [ ]:
for ik,p in enumerate(probs):
    k = dec(rev(binary(ik,5)))
    if p >0:
        print(k,p)

O sea, (8 ó 9) + 7 = 15 ó 16

Con dos bits inciertos:

In [ ]:
probs = adder4 @ tps([Id,erase,Id,Id,Id,Id,erase,Id,Id]) @ decode(ab)
probs

In [ ]:
for ik,p in enumerate(probs):
    k = dec(rev(binary(ik,5)))
    if p >0:
        print(k,p)

O sea, (8 ó 9) + (3 ó 7) = 11 ó 12 ó 15 ó 16

In [ ]:
plt.figure(figsize=(8,3))
showmat(adder4 @ tps([Id,erase,Id,Id,Id,Id,erase,Id,Id]))

## Reversible computation

Si la matriz tiene inversa significa que la computación se puede deshacer, del estado final se puede volver al de partida. La matriz de suma del ejemplo anterior claramente no es invertible a menos que nos las arreglemos para mantener las entradas, explícita o implícitameante en el resultado.

Afortunadamente existen juegos universales de puertas lógicas reversibles, lo cual implica que en principio se puede computar sin consumir energía. La que se haya consumido se recupera deshaciendo la operación.

Los circuitos lógicos reversibles se corresponden con matrices de permutación, tienen un uno en cada fila y columna.

## Quantum gates

CNOT: $(x,y) \rightarrow (x,x \oplus y)$

In [ ]:
plt.figure(figsize=(3,3))
showmat(CNot)

In [ ]:
for x,y in bits(2):
    print(x,y,encode(CNot@decode([x,y])))

Toffoli = CCNOT: $(x,y,z) \rightarrow (x,y,(x \land y) \oplus z )$

In [ ]:
plt.figure(figsize=(3,3))
showmat(Toffoli)

In [ ]:
for c1,c2,y in bits(3):
    print(c1,c2,y,encode(Toffoli@decode([c1,c2,y])))

Reversible And

In [ ]:
for x,y in bits(2):
    print(x,y,encode(Toffoli@decode([x,y,0])))

Reversible Or

In [ ]:
ROr = tps([Not,Not,Not]) @ Toffoli @ tps([Not,Not,Id])

In [ ]:
for x,y in bits(2):
    print(x,y,encode(ROr@decode([x,y,0])))

Reversible XOR:

In [ ]:
for x,y in bits(2):
    print(x,y,encode(Toffoli@decode([1,x,y])))

## Wiring

The logic gates must be "extended" to operate with all the variables in the circuit. The permutation matrices play the role of the wires connecting the inputs and outputs of each gate to the others.

In [ ]:
def permute(js,n):
    join  = Oper( [ [xs[k] for k in js] + [xs[k] for k in range(n) if k not in js] for xs in bits(n)] )
    return join

def extend(gate, tot):
    o,i = gate.shape
    n = round(np.log2(i))
    return tps([gate]+[Id]*(tot-n))

def operateWith(gate,js,n):
    join = permute(js, n)
    return join.T @ extend(gate, n) @ join

As an example, we compute $x \land y \land z$ using two Toffoli gates. We need two control inputs set to zero. We organize the computation so that the inputs keep the original order and the succesive zeros are changed to the intermediate results.

$$[x,y,z,0,0] \rightarrow [x,\, y,\, z,\, x \land y,\, x \land y \land z]$$

In [ ]:
And3 = operateWith(Toffoli,[2,3,4],5) @ operateWith(Toffoli,[0,1,3],5)

encode(And3 @ decode([1,1,1]+[0]*2))

In [ ]:
for x,y,z in bits(3):
    print(encode( And3 @ decode([x,y,z]+[0]*2)))

## Ancilla bits

Cada puerta necesita un bit constante auxiliar. Evidentemente, no tiene sentido introducir una nueva entrada por cada puerta lógica. Afortunadamente, dado que estamos utilizando computación reversible, podemos "descomputar" los resultados intermedios que no necesitemos con la operación inversa para recuperar la energía consumida y reutilizar los bits auxiliares o las entradas en operaciones posteriores.

In [ ]:
And3b = operateWith(Toffoli,[0,1,3],5).T @ operateWith(Toffoli,[2,3,4],5) @ operateWith(Toffoli,[0,1,3],5)

In [ ]:
for x,y,z in bits(3):
    print(encode( And3b @ decode([x,y,z]+[0]*2)))

## Deutchs-Jozsa

El ejemplo más simple de computación cuántica. Podemos determinar con una sola llamada si una función desconocida (tenemos su implementación oculta en una caja negra) que solo puede ser constante o "balanceada".

In [ ]:
# two WH gates in parallel for two bits
mix = tp(WH,WH)

def konst(x):
    return 1

def balanced(x):
    return 1 if x == 1 else 0

fun = balanced
#fun = konst

def xor(x,y):
    return 1 if x!=y else 0

# creates a reversible operation with an auxiliary input
reverK = Oper([( x, xor(y, konst(x)) ) for x,y in bits(2) ])

reverB = Oper([( x, xor(y, balanced(x)) ) for x,y in bits(2) ])

# check the operation and the order of bits
for x,y in bits(2):
    xs, yf = encode(reverK @ decode([x,y]))
    print (x,y, xs == x, yf == xor(y,konst(x)))
    xs, yf = encode(reverB @ decode([x,y]))
    print (x,y, xs == x, yf == xor(y,balanced(x)))

El primer bit de la salida nos da la solución: 0: constant, 1: balanced.

In [ ]:
# constant
amps = mix @ reverK @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
# balanced
amps = mix @ reverB @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
# with the identity in the auxiliary first bit, it remains uncertain
amps = tp(WH,Id) @ reverB @ mix @ decode([0,1])
print('Amplitudes:', amps)

probs = np.abs(amps)**2

print('probabilities:')
for k,v in zip(bits(2), probs):
    if v >0:
        print(k,v)

In [ ]:
showmat(reverK)

In [ ]:
showmat(reverB)

## Bernstein-Vazirani

Partimos de una función como caja negra que implementa el producto escalar mod 2 con un vector binario secreto.

In [ ]:
secret = [0,1,1,0]
n = len(secret)

def dot(ss,xs):
    return sum([s*x for s,x in zip(ss,xs)]) % 2

bervaz = Oper([ X + [xor(y, dot(secret,X))] for X in map(list,bits(n)) for y in bit])

showmat(bervaz)
bervaz.shape

Para determinar este vector mediante operaciones clásicas tenemos que evaluar la función una vez para cada elemento.

In [ ]:
for k in range(n):
    print(encode(bervaz @ decode(list(np.eye(4)[k].astype(int))+[0])))

Con puertas cuánticas podemos crear una superposición que obtiene el vector con una sola evaluación:

In [ ]:
amps = tps([WH]*(n+1)) @ (bervaz @ (tps([WH]*(n+1)) @ decode(([0]*(n)+[1]))))

In [ ]:
probs = np.abs(amps)**2
for k,v in zip(bits(n+1), probs):
    if v > 1/100:
        print(f'{k[:n]} | {k[-1]}   {100*v:.2f}%')

## Shor

La factorización de enteros se reduce a encontrar una raíz cuadrada modular no trivial de la unidad, que a su vez se reduce a encontrar el período de una secuencia.

Vamos a construir el circuito para $f(x)=a^x \mod N$

(Empezamos con un registro n=4 para comprobar los cálculos.)

In [ ]:
a = 13
N = 15

n = 4
q = 4

def f(x):
    r = a**x % N
    return r, binary(r,q)

In [ ]:
for k in range(2**n):
    print(k, f(k))

Se observa la periodicidad que el algoritmo tendrá que detectar.

Construimos el circuito que la implementa, que produce la misma entrada y el resultado de la función. En una implementación física real esto habría que hacerlo con puertas lógicas reversibles. Es la parte más complicada (ver *Reversible Adder* más abajo).

In [ ]:
expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])

In [ ]:
showmat(expmod)

Verificamos que funciona correctamente con la organizacion de bits establecida.

In [ ]:
bs = encode(expmod @ decode(binary(14,n)+[0]*q))
print(bs)
dec(bs[:n]), dec(bs[n:])

In [ ]:
bs = encode(expmod @ decode(binary(11,n)+[0]*q))
print(bs)
dec(bs[:n]), dec(bs[n:])

Alimentamos el circuito con una superposición de todas las entradas:

In [ ]:
amps = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
amps

Si observamos todos los bits del resultado, puede salir cualquier configuración de entrada con su salida asociada.

In [ ]:
def shprobs(amps,tol=1):
    probs = np.abs(amps)**2
    for k,v in zip(bits(n+q), probs):
        if v>tol/100:
            print(f'{dec(k[:n]):2} -> {dec(k[n:]):2}   {100*v:.2f}%')

In [ ]:
print('Probabilities:')
shprobs(amps)

Se obtienen exactamente las mismas probabilidades si se introduce un valor incierto clásico (usando el operadore `erase` anterior en vez de la puerta de Walsh-Hadamard). Esto significa que si desconocemos completamente qué entrada concreta se ha introducido, la salida puede ser cualquiera de las posibles con igual probabilidad.

En el caso cuántico se introduce un estado de superposición perfectamente definido y conocido, que se transforma, y al medirse en la base computacional se proyecta alguno de los resultados posibles.

In [ ]:
# Partial measurement of the bits in ks

def measure(state, ks):
    n = round(np.log2(len(state)))
    r = np.random.choice(np.arange(len(state)), p=np.abs(state)**2)
    print(r)
    xs = binary(r,n)
    print(xs)
    obs = np.array(xs)[ks]
    print(obs)
    newamps = np.array([ a if np.array_equal(np.array(bs)[ks] , obs) else 0 for bs, a in zip(bits(n), state) ])
    newamps = newamps/np.linalg.norm(newamps)
    return newamps

La primera idea clave del algoritmo de Shor es que al observar el valor de la función el estado de los qbits no observados, los que copian la entrada, queda en una superposición de los valores que producen este resultado concreto observado.

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

shprobs(collapsed)

En el caso clásico, esto nos diría que una de esas entradas es la que se introdujo concretamente en el circuito. En el caso cuántico tenemos un estado que mantiene todas las posibilidades. Si lo observamos obtendríamos una de ellas, igual que en el caso clásico.

Si de alguna manera pudiéramos medir estos qbits varias veces sin alterar el estado, obtendríamos diferentes valores con una sola ejecución de la exponenciación modular y podríamos deducir el período (la diferencia entre ellos es un múltiplo del período). Pero esto es físicamente imposible, no se puede clonar un estado cuántico. Habría que repetir el proceso ejecutando de nuevo la función desde el principio. En casos realistas de números grandes es muy improbable que se repita el resultado.

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

shprobs(collapsed)

La segunda clave del algoritmo de Shor es aplicar la transformada de Fourier a la parte del estado que contiene todas las entradas que producen el valor de salida observado, para determinar el período.

Hay que aumentar el número de qbits del registro que contiene la entrada para que se produzca un número suficiente de repeticiones. Se supone que debe ser $N^2 < 2^n < 2N^2$, pero en alguno de estos experimentos parece que funciona con valores menores.

In [ ]:
n = 6

expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])
amps = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
print('Probabilities')
shprobs(amps)

In [ ]:
collapsed = measure(amps, list(range(n,n+q)))

Queda una superposición de los valores de entrada que producen el mismo resultado:

In [ ]:
plt.rcParams["figure.figsize"] = [8,3]
plt.plot(collapsed);

In [ ]:
pos = np.where(abs(collapsed)>0.1)[0]
print(pos)
print(pos[1:] - pos[:-1])
(pos[1]-pos[0])/2**q

(El período en el espacio expandido va multiplicado por el tamaño del otro registro.)

Como comprobación, extraemos las amplitudes de las configuraciones no observadas.

In [ ]:
def showprobs2():
    sa = np.zeros(2**n)
    print('Probabilities:')
    for k,a in zip(bits(n+q), collapsed):
        x = dec(k[:n])
        v = np.abs(a)**2
        sa[x] += a
        if v >0:
            print(f'{x:3} -> {dec(k[n:]):3}   {100*v:.2f}%')

    plt.bar(np.arange(len(sa)),np.abs(sa),width=0.5);
    plt.xlabel('x'); plt.ylabel('amp');
    return sa

In [ ]:
sa = showprobs2()

The [Quantum Fourier Transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) aplica la TF a la secuencia de amplitudes de un estado cuántico, ordenadas de acuerdo con la enumeración binaria de los qubits... Se puede realizar físicamente con puertas de forma eficiente.

In [ ]:
def QFT(n):
    N = 2**n
    w = np.exp(1j*2*np.pi/N)
    r = np.array([[ w**(k*j) for k in range(N)] for j in range(N)]) / np.sqrt(N)
    return r

In [ ]:
abs(QFT(4)@np.conj(QFT(4).T) - np.eye(16)).max()

In [ ]:
showmat(np.hstack([np.real(QFT(5)),np.imag(QFT(5))]))

In [ ]:
plt.figure(figsize=(6,3))
pf = np.abs(QFT(n) @ sa)**2
plt.plot(pf);
np.where(pf>1/100)

Since the period probably will seldom be an exact divisor of the length we need the convergents (see below). We include here a simple implementation to compute the sequence of convergents of the continuous fraction expansion of a given fraction.

In [ ]:
from theonum import cf_expansion, convergents

In [ ]:
print('Probabilities:')
for j,v in enumerate(pf):
    if v > 1/100:
        cs = list(convergents(cf_expansion(j,2**n)))
        print(f'{100*v:6.2f}%   {j:3}  {cs}')

With the candidates we verify that we have found the modular square root of one. 

In [ ]:
a**2 % N, a**4 % N

And finally we obtain the factors:

In [ ]:
from math import gcd

r = 4
p = gcd(a**(r//2)-1, N)

p, N//p, N%p

Lo que ocurre se ve casi mejor en el espacio completo. Preparamos el circuito para otra factorización:

In [ ]:
a = 19
N = 21

n = 6
q = 5

for k in range(2*N):
    print(k, f(k))
print('...')

expmod =Oper([ xs + f(dec(xs))[1] for zs in bits(n+q) for xs in [list(zs[:n])]])

amps0 = expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q)))
print('Probabilities')
shprobs(amps0)

amps = tps([QFT(n)]+[Id]*q) @ (expmod @ (tps([WH]*n + [Id]*q) @ decode([0]*(n+q))))
print('\nWith QFT')
shprobs(amps,tol=1)

Repitiendo el experimento varias veces, aunque el valor de la función sea distinto, el resultado de la TF es siempre un múltiplo del período.

In [ ]:
probs0 = np.abs( measure(amps0, list(range(n,n+q))) )**2
plt.plot(probs0)
plt.show()
probs = np.abs( measure(amps, list(range(n,n+q))) )**2
plt.plot(probs);

In [ ]:
for k,v in zip(bits(n+q), probs):
    j = dec(k[:n])
    if v>5/100:
        cs = list(convergents(cf_expansion(j,2**n)))
        print(f'{100*v:6.2f}%   {j:3} - {dec(k[n:]):2}:  {cs}')
        for _,d in cs:
            if  a**d % N == 1:
                r = d
                break
print(r)
p = gcd(a**(r//2)-1, N)
p, N//p, N%p

## FFT for non-integer frequencies

In [ ]:
def shqft():
    x = np.zeros(256)
    x[5::9] = 1
    plt.plot(x);
    plt.title(f'period=9,  length=256,  true freq={256/9:.2f},  peaks={sum(x>0.5)}')
    plt.show()

    f = abs(np.fft.ifft(x))
    plt.plot(f,'.-');
    h = 0.04
    ks = list(np.where(f>h)[0])
    plt.plot([0,255],[h,h],color='gray',ls='dotted')
    sks = f'{ks[1:7]}'[1:-1]
    plt.title(f'FFT big peaks at {sks}, ...')
    return ks

In [ ]:
ks = shqft()

In [ ]:
[(k,list(convergents(cf_expansion(k,256)))) for k in ks]

## Reversible adder

The most complex part of Shor's algorithm is the circuit for modular exponentiation ([Pavlidis & Gizopoulos, 2014](https://arxiv.org/abs/1207.0511)). To get an idea we build an adder from scratch using just Toffoli gates.

The operation of a reversible one-bit full adder must be some something like

$$(x,y,c,0,\ldots) \rightarrow (\ldots,\; x\oplus y \oplus c,\;\, xy+xc+yc,\;\ldots)$$

where "$\ldots$" denotes auxiliary constant bits and intermediate results or inputs required for a reversible operation. A more or less direct translation of the above formulas may not be optimal. The following circuit by Feynman shown in the [Wikipedia article](https://en.wikipedia.org/wiki/Quantum_logic_gate) only requires two CCNOT and two CNOT gates (or three if we need B):

![Feynman adder](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Quantum_Full_Adder.png/350px-Quantum_Full_Adder.png)

In [ ]:
def op(g,*xs): return operateWith(g,[*xs],4)

FAdd = op(CNot,1,2) @ op(Toffoli,1,2,3) @ op(CNot,0,1) @ op(Toffoli,0,1,3)
showmat(FAdd)

for x,y,c in bits(3):
    print([x,y,c], encode(FAdd @ decode([x,y,c,0])))

Circuits for multiplier, divider, and powermod operations are left as an exercise ;)

## QFT from quantum gates

In [ ]:
# direct sum: if control bit=0 then A else B
def DSUM(A,B):
    na = len(A)
    nb = len(B)
    X = np.zeros([na+nb,na+nb],complex)
    X[:na,:na] = A
    X[-nb:,-nb:] = B
    return X

# phase shift
def R(n):
    return np.array([[1,0],[0,np.exp(2j*np.pi/2**n)]])

# controlled phase shift
def CR(n):
    return DSUM(Id,R(n))

# permutation matrix for bit reversal
def BitReversal(n):
    return  Oper([ list(reversed(bs)) for bs in bits(n) ])

showmat(BitReversal(3))

In [ ]:
def op(g,*xs): return operateWith(g,[*xs],5)

Q1 = op(CR(5),4,0) @ op(CR(4),3,0) @ op(CR(3),2,0) @ op(CR(2),1,0) @ op(WH,0)
Q2 = op(CR(4),4,1) @ op(CR(3),3,1) @ op(CR(2),2,1) @ op(WH,1)
Q3 = op(CR(3),4,2) @ op(CR(2),3,2) @ op(WH,2)
Q4 = op(CR(2),4,3) @ op(WH,3)
Q5 = op(WH,4)

Q = BitReversal(5) @ Q5 @ Q4 @ Q3 @ Q2 @ Q1
showmat(np.hstack([np.real(Q),np.imag(Q)]))

All operators are very sparse:

In [ ]:
showmat(op(WH,2))

In [ ]:
T  = op(CR(2),3,2)
showmat(np.hstack([np.real(T),np.imag(T)]))

## Tensors

Instead of this costly explicit representation of the operators over the full tensor product space of all state variables, we can work with proper tensors and apply simple operations along the required dimensions.

We use the utilities based on [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) described [here](tensolve.ipynb).

In [ ]:
import sympy # not used here but required by the next import
import umucv.tensor as tensor
from umucv.tensor import T

In [ ]:
zero = T([1,0])
one  = T([0,1])

H = T(np.array(
    [[1, 1],
    [1,-1]])/np.sqrt(2))

In [ ]:
qbit = T(2)*zero + T(5)*one
qbit

In [ ]:
state = zero('i') @ zero('j')
state

In [ ]:
H('iq') @ state('qj')

In [ ]:
H('jq') @ state('iq')

In [ ]:
H('il') @ H('jq') @ state('ql')

This is the Walsh-Hadamard transform in factorized form:

In [ ]:
H('il') @ H('jq')

The Toffoli universal reversible gate is a rank 6 tensor:

In [ ]:
Tof = T(np.array(
      [[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0]]).reshape(2,2,2,2,2,2))

for a in [zero('l'),one('l')]:
    for b in [zero('m'),one('m')]:
        r = Tof @ a @ b @ zero('n')
        print(encode(r.A.flatten()))

The composition of operators is just contractions of the connected input/output ports. The tensor product implementing the parallel combination of operations is not explicitly computed.

In [ ]:
Not = T(np.array(
      [[0, 1],
       [1, 0]]))

Or = Tof('abcpqn') @ Not('lp') @ Not('mq') @ Not('ia') @ Not('jb') @ Not('kc')

for a in [zero('l'),one('l')]:
    for b in [zero('m'),one('m')]:
        r = Or @ a @ b @ zero('n')
        print(encode(r.A.flatten()))

The inverse operation of the reversible gates is achieved just by changing the input/output role of the slots.

In [ ]:
state = zero('l') @ one('j') @ zero('n')
state

In [ ]:
result = state @ H('il') @ H('kn') @ Tof('rstijk')
result

In [ ]:
back = result @ H('li') @ H('nk') @ Tof('rstijk')
back

(The inverse operation of general unitary gates with complex components also requires conjugation.)

A few more experiments with the tensor representation of states and gates.

In [ ]:
def normalize(t):
    return t @ T(1/np.linalg.norm(t.A.flatten()))

def probs(st):
    """show the probabilities of the possible measurements of a state"""
    print(st.idx)
    for amp, reg in zip(st.A.flatten(),bits(len(st.A.shape))):
        p = amp * np.conj(amp)
        if p > 0.01:
            print(f"{p:.3f} {reg}")

In [ ]:
state = zero('a') @ zero('b') @ H('ka') @ one('c')
print(state)

In [ ]:
probs(state)

In [ ]:
CNOT =T(CNot.reshape(2,2,2,2))

for a in [zero('k'),one('k')]:
    for b in [zero('l'),one('l')]:
        r = CNOT @ a @ b
        print(encode(r.A.flatten()))

In the Bell states the bits are entagled: the state cannot be factorized:

In [ ]:
bell = zero('a') @ zero('b') @ H('ka') @ CNOT('ijkb')
bell

In [ ]:
probs(bell)

A reversible 1-bit full adder in tensor form:

In [ ]:
ADD = T(FAdd.reshape(2,2,2,2,2,2,2,2))
print(ADD.idx)

In [ ]:
def tb(t): return encode(t.A.flatten())

for a in [zero('m'),one('m')]:
    for b in [zero('n'),one('n')]:
        for c in [zero('o'),one('o')]:
            r = ADD @ a @ b @ c @ zero('p')
            print([tb(a),tb(b),tb(c)], r.idx, encode(r.A.flatten()))

Ojo: es reversible pero no para cualquier selección de entradas/salidas. Algunas son ambiguas y otras imposibles. La reordenación del tensor da lugar a una matriz que no es una permutación y la norma del estado resultante no es uno.

We can build multibit adders as a sequence of 16x16 tensor gates, acting on a bigger space.  

In [ ]:
#        11 + 7
bs = '0 1011 0111 0000'.replace(' ','')
js = 'm dcba hgfe ijkl'.replace(' ','')

lstate = [ (zero if b=='0' else one)(k) for k,b in zip(js,bs) ]
state = tensor.prod(lstate)
state.idx

In [ ]:
state.A.size, round(np.sqrt(ADD.A.size))

TODO: include diagram to make sense of the indexes.

In [ ]:
ADDER_4 = [ ADD('nopqaemi'), ADD('rstubfqj'), ADD('vwxycguk'), ADD('zABCdhyl') ]

result = tensor.prod( [state] + ADDER_4 )
probs(result.reorder('zvrnAwsoCBxtp'))

(The tensor is reordered to show the sum in the last five bits.)

We can operate on superpositions:

In [ ]:
# (10_11) + 7 --> 17_18
result = tensor.prod ( [state('Xbcdefghijklm') @ H('aX')] + ADDER_4 )
probs(result.reorder('zvrnAwsoCBxtp'))

With a Bell state we have two inputs with correlated bits (opposite in this case):

In [ ]:
#    10_11      7_6
state_b = state('XbcdYfghijklm') @ H('UX') @ CNOT('aeUY')
probs(state_b.reorder(js))

Which produce the same final result:

In [ ]:
result = tensor.prod ( [state_b] + ADDER_4 )
probs(result.reorder('zvrnAwsoCBxtp'))

We introduce another superposition in the intial carry bit. We copy it to another qubit because this input is not preserved in the adder output, and we want to measure it.  

In [ ]:
result = result = tensor.prod ( [state_b('abcdefghijklX') @ H('MX') @ zero('E') @ CNOT('mNME')] + ADDER_4 )
probs(result.reorder('NzvrnAwsoCBxtp'))

In order to simulate a partial readout we must set to zero the amplitudes of the possibilities inconsistent with the measurement. This projection can be done with explicit assignments, but the most natural way is just a contraction with the observed partial state. For instance, if we observe the qbit which contains the superposition of the initial carry bit (index "m" in this case) and see a "1", the state collapses to:

In [ ]:
m2 = normalize(result @ one('N'))
probs(m2.reorder('zvrnAwsoCBxtp'))

If we instead osbserve the least significant bit of the result and it happens to be a "1", then the observation of the copy of the initial carry will be '0'.

In [ ]:
m2 = normalize(result @ one('p'))
probs(m2.reorder('NzvrnAwsoCBxt'))

A partial readout on a Bell state:

In [ ]:
probs(bell)

In [ ]:
probs(normalize(bell @ one('j')))

In [ ]:
probs(normalize(bell @ zero('i')))

The tensor formalism is particularly nice for this kind of computations. La contracción es una generalización de la operación de  indexado de un array.

Volviendo al sumador, podemos operar con una superposición de todas las posibilidades:

In [ ]:
def mkState(bs,js):
    lstate = [ (zero if b=='0' else one)(k) for k,b in zip(js,bs) ]
    return tensor.prod(lstate)

In [ ]:
NA = mkState('0000','ABCD') @ H('aA') @ H('bB') @ H('cC') @ H('dD')
NB = mkState('0101','hgfe')
NZ = mkState('0000','ijlk')

In [ ]:
m3 = tensor.prod( [NA, NB, NZ, zero('m')] + ADDER_4 )
probs(m3.reorder('zvrnAwsoCBxtp'))

We get subtraction for free:

In [ ]:
NS = mkState('11001','CBxtp')  # 25
NA = mkState('1011','dcba')    # 11
NZ = mkState('0000','ijkl') @ zero('m')

m4 = tensor.prod( [NA, NS, NZ] + ADDER_4[::-1] )
print(m4.idx, len(m4.idx), np.linalg.norm(m4.A.flatten()))
probs(m4.reorder('Anorsvwzhgfe'))

In [ ]:
NS = mkState('01101','CBxtp')  # 13
NA = mkState('1011','dcba')    # 11
NZ = mkState('0000','ijkl') @ one('m')

m4 = tensor.prod( [NA, NS, NZ] + ADDER_4[::-1] )
print(m4.idx, len(m4.idx), np.linalg.norm(m4.A.flatten()))
probs(m4.reorder('Anorsvwzhgfe'))

Es necesario establecer el initial carry porque si no la operación no es invertible con las entradas / salidas establecidas. Además, hay operaciones imposibles:

In [ ]:
NS = mkState('00101','CBxtp')  # 5
NA = mkState('0111','dcba')    # 7
NZ = mkState('0000','ijkl') @ zero('m')

m4 = tensor.prod( [NA, NS, NZ] + ADDER_4[::-1] )
print(m4.idx, len(m4.idx), np.linalg.norm(m4.A.flatten()))
probs(m4.reorder('Anorsvwzhgfe'))

In [ ]:
NS = mkState('01101','CBxtp')  # 13
NA = mkState('1011','dcba')    # 11
NZ = mkState('0000','ijkl')

m4 = tensor.prod( [NA, NS, NZ] + ADDER_4[::-1] )
print(m4.idx, len(m4.idx), np.linalg.norm(m4.A.flatten()))
probs(m4.reorder('mAnorsvwzhgfe'))

Esto significa, creo, que aunque el tensor matemático permita elegir las entradas / salidas y, de alguna manera, resuelve lo mejor posible la operación, la puerta física solo es unitaria para unas entradas / salidas predeterminadas.